In [1]:
from pymongo import MongoClient
import pandas as pd
import numpy as np
import folium

In [2]:
def connectCollection(database, collection):
    client = MongoClient()
    db = client[database]
    coll = db[collection]
    return db, coll

In [3]:
db, coll = connectCollection('companies','companies')

In [4]:
db

Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'companies')

In [5]:
coll

Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'companies'), 'companies')

In [6]:
# companies = list(coll.find()) # 18801

# Query en compass: {$or:[{"offices.latitude":{$ne:null}},{"offices.longitude":{$ne:null}}]}
# En python es None en lugar de null.

companies = list(coll.find({'$or':[{"offices.latitude":{'$ne':None}},{"offices.longitude":{'$ne':None}}]})) # 14082

In [7]:
len(companies) # 14082

14082

In [8]:
companies[0]

{'_id': ObjectId('52cdef7c4bab8bd675297d8b'),
 'name': 'AdventNet',
 'permalink': 'abc3',
 'crunchbase_url': 'http://www.crunchbase.com/company/adventnet',
 'homepage_url': 'http://adventnet.com',
 'blog_url': '',
 'blog_feed_url': '',
 'twitter_username': 'manageengine',
 'category_code': 'enterprise',
 'number_of_employees': 600,
 'founded_year': 1996,
 'deadpooled_year': 2,
 'tag_list': '',
 'alias_list': 'Zoho ManageEngine ',
 'email_address': 'pr@adventnet.com',
 'phone_number': '925-924-9500',
 'description': 'Server Management Software',
 'created_at': datetime.datetime(2007, 5, 25, 19, 24, 22),
 'updated_at': 'Wed Oct 31 18:26:09 UTC 2012',
 'overview': '<p>AdventNet is now <a href="/company/zoho-manageengine" title="Zoho ManageEngine" rel="nofollow">Zoho ManageEngine</a>.</p>\n\n<p>Founded in 1996, AdventNet has served a diverse range of enterprise IT, networking and telecom customers.</p>\n\n<p>AdventNet supplies server and network management software.</p>',
 'image': {'avail

In [9]:
def set_key(dictionary, key, value):
    if key not in dictionary:
        dictionary[key] = value
    elif type(dictionary[key]) == list:
        dictionary[key].append(value)
    else:
        dictionary[key] = [dictionary[key], value]

In [10]:
companies_dict = {}
for i in range(len(companies)):
    set_key(companies_dict, '_id', companies[i]['_id'])
    set_key(companies_dict, 'name', companies[i]['name'])
    set_key(companies_dict, 'founded_year', companies[i]['founded_year'])
    set_key(companies_dict, 'category_code', companies[i]['category_code'])
    set_key(companies_dict, 'total_money_raised', companies[i]['total_money_raised'])
    set_key(companies_dict, 'deadpooled_year', companies[i]['deadpooled_year'])

In [11]:
# print(len(companies_dict['id'])) # 14082
# print(len(companies_dict['name'])) # 14082
# print(len(companies_dict['founded_year'])) # 14082
# print(len(companies_dict['category_code'])) # 14082
# print(len(companies_dict['total_money_raised'])) # 14082
# print(len(companies_dict['deadpooled_year'])) # 14082
# print(companies_dict.keys())

In [12]:
companies_df = pd.DataFrame.from_dict(companies_dict)

In [13]:
companies_df.head(10)

,_id,name,founded_year,category_code,total_money_raised,deadpooled_year
0,52cdef7c4bab8bd675297d8b,AdventNet,1996.0,enterprise,$0,2.0
1,52cdef7c4bab8bd675297d8c,Zoho,2005.0,software,$0,3.0
2,52cdef7c4bab8bd675297d8a,Wetpaint,2005.0,web,$39.8M,1.0
3,52cdef7c4bab8bd675297d8d,Digg,2004.0,news,$45M,NaN
4,52cdef7c4bab8bd675297d91,Geni,2006.0,web,$16.5M,NaN
5,52cdef7c4bab8bd675297d90,Postini,1999.0,web,$0,NaN
6,52cdef7c4bab8bd675297d92,Flektor,NaN,games_video,$0,NaN
7,52cdef7c4bab8bd675297d93,Fox Interactive Media,1979.0,web,$0,NaN
8,52cdef7c4bab8bd675297d96,Gizmoz,2003.0,web,$18.1M,NaN
9,52cdef7c4bab8bd675297d98,Slacker,2006.0,music,$73.1M,NaN


In [14]:
num_offices = []
for i in range(len(companies)):
    num_offices.append(len(companies[i]['offices']))
# print(len(num_offices)) # 14082
# max(num_offices) # 12
companies_df['num_offices'] = num_offices

In [15]:
for i in range(max(num_offices)):
    companies_df[f'office_{i+1}_longitude'] = [np.nan] * len(num_offices)
    companies_df[f'office_{i+1}_latitude'] = [np.nan] * len(num_offices)
    companies_df[f'office_{i+1}_location'] = [{}] * len(num_offices)
    companies_df[f'office_{i+1}_city'] = [None] * len(num_offices)
    companies_df[f'office_{i+1}_state_code'] = [None] * len(num_offices)
    companies_df[f'office_{i+1}_country_code'] = [None] * len(num_offices)

In [16]:
companies_df.dtypes

_id                        object
name                       object
founded_year              float64
category_code              object
total_money_raised         object
deadpooled_year           float64
num_offices                 int64
office_1_longitude        float64
office_1_latitude         float64
office_1_location          object
office_1_city              object
office_1_state_code        object
office_1_country_code      object
office_2_longitude        float64
office_2_latitude         float64
office_2_location          object
office_2_city              object
office_2_state_code        object
office_2_country_code      object
office_3_longitude        float64
office_3_latitude         float64
office_3_location          object
office_3_city              object
office_3_state_code        object
office_3_country_code      object
office_4_longitude        float64
office_4_latitude         float64
office_4_location          object
office_4_city              object
office_4_state

In [17]:
companies_data = []
for i in range(len(companies)):
    if len(companies[i]['offices'])==0:
        companies_data.append([[0, np.nan, np.nan, np.nan, np.nan, np.nan]])
    elif len(companies[i]['offices'])>0:
        aux_offices = []
        for j in range(len(companies[i]['offices'])):
            aux_offices.append([j+1, companies[i]['offices'][j]['latitude'], companies[i]['offices'][j]['longitude'],
                                companies[i]['offices'][j]['city'], companies[i]['offices'][j]['state_code'],
                                companies[i]['offices'][j]['country_code']])
        companies_data.append(aux_offices)
len(companies_data)

14082

In [18]:
for i in range(len(companies_data)):
    for j in range(len(companies_data[i])):
        if companies_data[i][j][0] != 0:
            companies_df.at[i, f'office_{j+1}_longitude'] = companies_data[i][j][2]
            companies_df.at[i, f'office_{j+1}_latitude'] = companies_data[i][j][1]
            companies_df.at[i, f'office_{j+1}_city'] = companies_data[i][j][3]
            companies_df.at[i, f'office_{j+1}_state_code'] = companies_data[i][j][4]
            companies_df.at[i, f'office_{j+1}_country_code'] = companies_data[i][j][5]

In [19]:
def getLocation(longitude,latitude):
    loc = {
        'type':'Point',
        'coordinates':[longitude, latitude]
    }
    return loc

In [20]:
for i in range(len(companies_data)):
    for j in range(len(companies_data[i])):
        if companies_data[i][j][0] != 0:
            companies_df.at[i, f'office_{j+1}_location'] = getLocation(companies_df.at[i, f'office_{j+1}_longitude'],
                                                                       companies_df.at[i, f'office_{j+1}_latitude'])

In [21]:
companies_df

,_id,name,founded_year,category_code,total_money_raised,deadpooled_year,num_offices,office_1_longitude,office_1_latitude,office_1_location,...,office_11_location,office_11_city,office_11_state_code,office_11_country_code,office_12_longitude,office_12_latitude,office_12_location,office_12_city,office_12_state_code,office_12_country_code
0,52cdef7c4bab8bd675297d8b,AdventNet,1996.0,enterprise,$0,2.0,1,-121.904945,37.692934,"{'type': 'Point', 'coordinates': [-121.904945,...",...,{},None,None,None,NaN,NaN,{},None,None,None
1,52cdef7c4bab8bd675297d8c,Zoho,2005.0,software,$0,3.0,1,-121.904945,37.692934,"{'type': 'Point', 'coordinates': [-121.904945,...",...,{},None,None,None,NaN,NaN,{},None,None,None
2,52cdef7c4bab8bd675297d8a,Wetpaint,2005.0,web,$39.8M,1.0,2,-122.333253,47.603122,"{'type': 'Point', 'coordinates': [-122.333253,...",...,{},None,None,None,NaN,NaN,{},None,None,None
3,52cdef7c4bab8bd675297d8d,Digg,2004.0,news,$45M,NaN,1,-122.394523,37.764726,"{'type': 'Point', 'coordinates': [-122.394523,...",...,{},None,None,None,NaN,NaN,{},None,None,None
4,52cdef7c4bab8bd675297d91,Geni,2006.0,web,$16.5M,NaN,1,-118.393064,34.090368,"{'type': 'Point', 'coordinates': [-118.393064,...",...,{},None,None,None,NaN,NaN,{},None,None,None
5,52cdef7c4bab8bd675297d90,Postini,1999.0,web,$0,NaN,1,-122.247573,37.506885,"{'type': 'Point', 'coordinates': [-122.247573,...",...,{},None,None,None,NaN,NaN,{},None,None,None
6,52cdef7c4bab8bd675297d92,Flektor,NaN,games_video,$0,NaN,1,-118.379768,34.025958,"{'type': 'Point', 'coordinates': [-118.379768,...",...,{},None,None,None,NaN,NaN,{},None,None,None
7,52cdef7c4bab8bd675297d93,Fox Interactive Media,1979.0,web,$0,NaN,1,-118.394170,34.076179,"{'type': 'Point', 'coordinates': [-118.39417, ...",...,{},None,None,None,NaN,NaN,{},None,None,None
8,52cdef7c4bab8bd675297d96,Gizmoz,2003.0,web,$18.1M,NaN,1,-122.169472,37.484130,"{'type': 'Point', 'coordinates': [-122.169472,...",...,{},None,None,None,NaN,NaN,{},None,None,None
9,52cdef7c4bab8bd675297d98,Slacker,2006.0,music,$73.1M,NaN,1,-117.081406,33.022176,"{'type': 'Point', 'coordinates': [-117.081406,...",...,{},None,None,None,NaN,NaN,{},None,None,None


In [22]:
companies_df.total_money_raised.replace('$0','0', inplace=True)
companies_df['total_money_raised_currency'] = [None] * len(num_offices)

currencies = []
for i in range(len(companies_df)):
    currencies.append(companies_df.total_money_raised[i][0])
set(currencies)

{'$', '0', 'C', 'k', '£', '¥', '€'}

In [23]:
for i in range(len(companies_df)):
    if companies_df.at[i,'total_money_raised'] == '0':
        companies_df.at[i,'total_money_raised_currency'] = '$'
    else:
        companies_df.at[i,'total_money_raised_currency'] = companies_df.total_money_raised[i][0]
        companies_df.at[i,'total_money_raised'] = companies_df.total_money_raised[i][1:]

In [24]:
for i in range(len(companies_df)):
    if companies_df.at[i,'total_money_raised'][0] == '$':
        companies_df.at[i,'total_money_raised_currency'] = companies_df.total_money_raised[i][0]
        companies_df.at[i,'total_money_raised'] = companies_df.total_money_raised[i][1:]

In [25]:
companies_df.total_money_raised.replace('r21M','21M',inplace=True)

In [26]:
for i in range(len(companies_df)):
    if companies_df.total_money_raised[i][-1] == 'M':
        companies_df.at[i,'total_money_raised'] = str(float(companies_df.total_money_raised[i].split('M')[0])*1000000)
    elif companies_df.total_money_raised[i][-1] == 'B':
        companies_df.at[i,'total_money_raised'] = str(float(companies_df.total_money_raised[i].split('B')[0])*1000000000)
    elif companies_df.total_money_raised[i][-1] == 'k':
        companies_df.at[i,'total_money_raised'] = str(float(companies_df.total_money_raised[i].split('k')[0])*1000)

In [27]:
print([e for e in companies_df.total_money_raised])

['0', '0', '39800000.0', '45000000.0', '16500000.0', '0', '0', '0', '18100000.0', '73100000.0', '44200000.0', '25800000.0', '0', '5580000.0', '1160000000.0', '0', '2430000000.0', '45000000.0', '0', '28300000.0', '13200000.0', '22500000.0', '32100000.0', '6700000.0', '0', '21000000.0', '23400000.0', '69800000.0', '1000000.0', '0', '8400000.0', '5200000.0', '88700000.0', '5000000.0', '0', '18300000.0', '4700000.0', '9000000.0', '0', '178000000.0', '120000000.0', '33000000.0', '2500000.0', '11500000.0', '0', '3000000.0', '14700000.0', '24000000.0', '11000000.0', '4000000.0', '0', '15000000.0', '52500000.0', '0', '9000000.0', '0', '100000.0', '0', '6800000.0', '7000000.0', '4990000.0', '3100000.0', '12700000.0', '32200000.000000004', '0', '12500000.0', '5000000.0', '0', '0', '18000000.0', '25000000.0', '0', '0', '4000000.0', '0', '450000.0', '0', '0', '90000000.0', '1000000.0', '225000.0', '11200000.0', '0', '50000.0', '32000000.0', '20600000.0', '0', '0', '4099999.9999999995', '32000000.0

In [28]:
companies_df['total_money_raised'] = companies_df['total_money_raised'].astype('float')
companies_df['total_money_raised'].dtype

dtype('float64')

In [29]:
print([e for e in companies_df.total_money_raised])

[0.0, 0.0, 39800000.0, 45000000.0, 16500000.0, 0.0, 0.0, 0.0, 18100000.0, 73100000.0, 44200000.0, 25800000.0, 0.0, 5580000.0, 1160000000.0, 0.0, 2430000000.0, 45000000.0, 0.0, 28300000.0, 13200000.0, 22500000.0, 32100000.0, 6700000.0, 0.0, 21000000.0, 23400000.0, 69800000.0, 1000000.0, 0.0, 8400000.0, 5200000.0, 88700000.0, 5000000.0, 0.0, 18300000.0, 4700000.0, 9000000.0, 0.0, 178000000.0, 120000000.0, 33000000.0, 2500000.0, 11500000.0, 0.0, 3000000.0, 14700000.0, 24000000.0, 11000000.0, 4000000.0, 0.0, 15000000.0, 52500000.0, 0.0, 9000000.0, 0.0, 100000.0, 0.0, 6800000.0, 7000000.0, 4990000.0, 3100000.0, 12700000.0, 32200000.000000004, 0.0, 12500000.0, 5000000.0, 0.0, 0.0, 18000000.0, 25000000.0, 0.0, 0.0, 4000000.0, 0.0, 450000.0, 0.0, 0.0, 90000000.0, 1000000.0, 225000.0, 11200000.0, 0.0, 50000.0, 32000000.0, 20600000.0, 0.0, 0.0, 4099999.9999999995, 32000000.0, 136000000.0, 0.0, 2000000.0, 0.0, 15000000.0, 78000000.0, 5000000.0, 0.0, 0.0, 0.0, 10000000.0, 8000000.0, 13800000.0, 26

In [30]:
companies_df.deadpooled_year.replace(1.0,2006,inplace=True) # 2005 + 1
companies_df.deadpooled_year.replace(2.0,1998,inplace=True) # 1996 + 2
companies_df.deadpooled_year.replace(3.0,2008,inplace=True) # 2005 + 3

In [33]:
set(companies_df.total_money_raised_currency)

{'$', 'k', '£', '¥', '€'}

In [34]:
companies_df.total_money_raised_currency.replace('k','SEK',inplace=True)
companies_df.total_money_raised_currency.replace('$','USD',inplace=True)
companies_df.total_money_raised_currency.replace('£','GBP',inplace=True)
companies_df.total_money_raised_currency.replace('¥','JPY',inplace=True)
companies_df.total_money_raised_currency.replace('€','EUR',inplace=True)

In [35]:
companies_df.to_csv('./input/companies_df.csv', index=False)

In [36]:
companies_df_final = companies_df[companies_df.num_offices>0]

In [37]:
len(companies_df_final) # 9025

9025

In [38]:
def exchangerate_api_request(currency):
    '''
    Requests to exchangerate-api
    currency must be a string of three capital letters: e.g. EUR
    '''
    import requests
    url = "https://api.exchangerate-api.com/v4/latest/{}".format(currency)
    res = requests.get(url)
    return res

In [39]:
exchangerateGBP = exchangerate_api_request('GBP').json()
exchangerateSEK = exchangerate_api_request('SEK').json()
exchangerateEUR = exchangerate_api_request('EUR').json()

In [70]:
print(exchangerateGBP['rates']['USD']) # multiplicar la columna GBP por esto
print(exchangerateSEK['rates']['USD']) # multiplicar la columna SEK por esto
print(exchangerateEUR['rates']['USD']) # multiplicar la columna EUR por esto

1.288341
0.103431
1.102924


In [ ]:
companies_df_final['total_money_raised_USD'] = [0.0]*len(companies_df_final)

In [ ]:
# 550, 609, 680, 716, 742, 941, 947, 948, 959, 968, 989, 1000, 1004, 1006, 1007, 1009...
for i in range(len(companies_df_final)):
    if companies_df_final.at[i,'total_money_raised_currency'] == 'USD':
        companies_df_final.at[i,'total_money_raised_USD'] = companies_df_final.at[i,'total_money_raised']
    elif companies_df_final.at[i,'total_money_raised_currency'] == 'GBP':
        companies_df_final.at[i,'total_money_raised_USD'] = companies_df_final.at[i,'total_money_raised']*exchangerateGBP['rates']['USD']   
    elif companies_df_final.at[i,'total_money_raised_currency'] == 'SEK':
        companies_df_final.at[i,'total_money_raised_USD'] = companies_df_final.at[i,'total_money_raised']*exchangerateSEK['rates']['USD']   
    elif companies_df_final.at[i,'total_money_raised_currency'] == 'EUR':
        companies_df_final.at[i,'total_money_raised_USD'] = companies_df_final.at[i,'total_money_raised']*exchangerateEUR['rates']['USD']     

In [ ]:
# money_to_USB = [0,0,39800000,45000000,16500000,0,0,0,18100000,73100000,44200000,25800000,0,5580000,1160000000,0,2430000000,45000000,0,28300000,13200000,22500000,32100000,6700000,0,21000000,23400000,69800000,1000000,0,8400000,5200000,88700000,5000000,0,18300000,4700000,9000000,0,178000000,120000000,33000000,2500000,11500000,0,3000000,14700000,24000000,11000000,4000000,0,15000000,52500000,0,9000000,0,100000,0,6800000,7000000,4990000,3100000,12700000,32200000,0,12500000,5000000,0,0,18000000,25000000,0,0,4000000,0,450000,0,0,90000000,1000000,248157.9,11200000,0,50000,32000000,20600000,0,0,4100000,32000000,136000000,0,2000000,0,15000000,78000000,5000000,0,0,0,10000000,8000000,13800000,26000000,46000000,2000000,0,1000000,0,40400000,7500000,1250000,1600000,0,10000000,140000000,409000000,16500000,0,0,200000000,6500000,5000000,110000000,66000000,0,6200000,5600000,96800000,0,16000000,14300000,5000000,31200000,0,0,23100000,44000000,22300000,0,81300000,0,0,0,0,0,10000000,0,8500000,0,7180000,11000000,0,0,12700000,300000,55600000,131000000,46800000,0,40100000,0,52000000,0,0,0,0,17500000,21500000,0,2700000,28700000,1500000,5000000,0,0,1750000,0,5000000,7300000,12683626,44700000,0,0,0,2000000,0,0,0,34000000,1300000,0,2060000,0,0,7500000,12000000,0,1750000,0,31100000,0,0,1000000,0,515000,100000,20000000,0,10600000,315000,11900000,300000,4000000,0,82700000,1500000,0,0,145000000,0,5500000,8000000,0,0,0,0,197000000,9000000,1500000,600000,0,43500000,0,0,29000000,0,315000,70000000,0,0,1310000,14500000,1000000000,0,0,0,0,0,0,65000000,0,0,4500000,4800000,75300000,0,0,0,5500000,12700000,0,34000000,50000000,0,0,3200000,28300000,36100000,386502.3,10000,0,0,0,0,0,0,0,1310000,670000,0,22600000,19000000,40800000,4500000,8500000,455000,15000,1920000,415000,0,1420000,0,0,1950000,18300000,24000000,0,9000000,6000000,33900000,4200000,0,5000000,8000000,11500000,2000000,0,63500000,0,0,0,5750000,0,27000000,7000000,0,0,0,40000000,2250000,26500000,4000000,3308772,4000000,52200000,12600000,48900000,17000000,39300000,13300000,3000000,0,22900000,11000000,78600000,33000000,48500000,113000000,11600000,0,0,17600000,38000000,0,0,255000000,0,6700000,0,0,53200000,0,0,0,3000000,45000000,0,0,0,14000000,0,0,20900000,0,0,0,48000000,0,0,5514620,0,10500000,0,0,248000000,7900000,0,0,12000000,75000000,175000,850000,7000000,0,5800000,7000000,0,6500000,28300000,10000000,0,19000000,0,4000000,0,41500000,127000000,7000000,0,0,0,0,12700000,101000000,4000000,0,13500000,0,0,550000,0,1150000,0,0,0,0,54000000,49000000,26800000,0,12300000,0,7500000,0,7400000,4800000,6500000,0,0,4000000,0,6300000,0,100000000,0,40800000,46800000,24800000,25200000,0,2800000,0,0,0,1830000,1900000,3500000,0,13100000,0,169000,0,50000,7100000,250000,0,0,1000000,0,0,0,0,3970000,0,450000,47500000,0,8000000,5000000,24500000,0,24500000,32400000,23000000,8700000,0,27300000,1000000,31000000,400000,1500000,1750000,32000000,0,0,0,0,46400000,0,0,0,2500000,2000000,33000000,3000000,32400000,0,0,11600000,0,0,0,0,500000,7350000,9400000,403000000,3110000,25100000,8000000,0,0,0,850000,1200000,100000,0,41800000,2300000,0,0,0,0,0,800000,0,0,41000000,0,0,0,17600000,19300000,135000,10000000,96600000,0,25000000,11000000,44000000,4000000,2600000,250000,116000000,2500000,16000000,0,0,0,14500000,2500000,5000000,3000000,0,0,0,0,0,46400000,0,0,0,17900000,48528656,0,0,0,4000000,0,650000,200000,16000000,1010000,0,16800000,0,0,22100000,161000000,2550000,72900000,0,17800000,35800000,0,21600000,4000000,3400000,109000000,0,30600000,24000000,125000000,0,65000000,8300000,16600000,14000000,0,10000000,8500000,0,0,12000000,0,0,0,0,200000,300000,4300000,0,55800000,20000000,39800000,0,0,2500000,6850000,2250000,32000000,21500000,1000000,56500000,0,15000000,27800000,69800000,11000000,30000000,0,0,35000000,0,1000000,10500000,600000,22400000,0,0,0,35700000,4080818.8,37000000,0,0,1500000,0,8950000,0,50000,0,3349686.6,2500000,13300000,3600000,4800000,0,21300000,11000000,0,0,33000000,0,900000,7900000,9020000,1000000,0,800000,5000000,750000,150000,32000000,5100000,1000000,40800000,10000000,0,0,0,0,3100000,0,25000000,0,0,10000000,22600000,1000000,0,75000000,15300000,46800000,0,0,0,0,1950000,0,586000000,0,24100000,0,0,920000,0,0,50000000,0,350000,29900000,2000000,1500000,14000000,0,10400000,70000,5000000,0,800000,0,0,0,1200000,0,15600000,40000000,500000,0,11000000,0,25000000,5000000,500000,8500000,0,0,10000,1000000,0,7000000,0,6700000,0,7000000,2500000,12000000,17200000,40000000,5000000,149000000,1102924,5400000,2500000,0,19400000,4000000,500000,0,3500000,11000000,25900000,141000000,131000000,0,10500000,500000,16000000,0,0,0,0,3000000,23000000,6000000,50200000,4500000,0,10000000,38602340,17000000,7000000,10200000,0,84100000,0,84000000,7000000,0,1102924,14000000,0,236000000,715000,21700000,0,11000000,55000000,0,30500000,8000000,0,45500000,0,0,107000000,5000000,2500000,0,600000,1764678.4,0,14800000,0,96500000,36500000,1000000,68500000,20000000,0,17500000,23300000,285000000,253000000,123000000,298000000,91700000,16400000,14500000,0,520000000,2500000,1000000,0,0,0,0,12500000,823000000,5700000,1250000,0,9650000,0,5600000,0,0,0,0,0,0,0,10000000,10000000,0,10000000,60200000,100000,0,0,0,80000000,47000000,0,3000000,0,135000000,9150000,6000000,0,500000,0,15600000,42000000,23300000,19400000,0,268000,0,0,1500000,112000000,5700000,0,0,51300000,14500000,34000000,58800000,4000000,210000000,0,35000000,0,11000000,0,0,500000,9250000,1300000,5000000,13000000,4500000,322085.25,3000000,6000000,0,0,0,0,0,0,1102924,0,0,0,6286666.8,10433661.04,3600000,60000000,3300000,90000000,13500000,0,50000000,0,12500000,1000000,0,28300000,0,0,65000000,1510000,13000000,3150000,0,0,68200000,0,0,0,1000000,0,0,900000,0,0,2000000,2000000,5000000,53000000,0,0,20000000,0,0,0,0,0,5000000,0,0,0,0,0,0,0,0,0,0,0,0,0,56600000,0,0,0,0,0,0,13500000,0,320000,0,0,0,0,0,0,0,0,0,2700000,0,0,0,0,12200000,0,0,0,0,0,5800000,100000,0,0,0,0,12000000,12000000,0,7300000,0,8990000,355000,0,0,11000000,0,0,0,0,0,0,990000,0,0,0,850000,0,23500000,0,2230000,0,500000,0,0,0,0,0,0,3500000,4000000,0,0,0,0,20000,0,0,10000000,50000,10800000,0,0,9500000,200000,200000,7000000,0,0,100000,220584.8,0,0,0,0,0,0,4000000,0,0,0,0,1000000,0,0,6630000,0,7050000,300000,0,26000000,0,0,0,7250000,0,0,0,0,0,0,0,0,0,230000000,0,0,0,0,0,0,0,0,0,0,0,0,17100000,118000000,0,0,0,0,0,10000000,0,100000,0,22000000,0,0,0,0,2080000,0,0,0,0,750000,2630000,0,0,0,15600000,7500000,0,50000,0,0,0,0,0,0,500000,16700000,0,21000000,0,2000000,0,0,5000000,0,20955556,9700000,4440000,0,26000000,0,0,0,0,0,0,8110000,251000000,20000,0,0,0,0,27000000,11500000,0,0,0,0,17100000,17600000,11139532.4,29300000,0,0,6700000,1500000,16000000,34500000,0,1000000,0,0,0,0,6540000,0,0,0,0,0,0,11200000,0,10000000,0,6250000,0,0,11000000,0,0,300000,0,61000000,0,56300000,0,9000000,0,92500000,0,0,0,0,41000000,0,0,35000000,9010000,0,0,0,1830000,0,0,750000,0,0,0,28500000,0,0,0,0,2000000,0,0,0,0,0,0,0,4250000,19600000,0,40500000,45000000,0,0,0,0,0,0,0,0,0,7000000,139000000,15400000,200000,0,0,0,0,0,0,0,0,0,60000,0,0,42500000,0,0,0,0,0,0,3000000,42700000,0,11500000,10000000,10000000,0,0,35000000,0,0,0,0,0,0,0,30300000,10300000,8000000,12000000,0,94800000,0,0,2200000,0,0,0,0,16300000,51400000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,10000000,0,1900000,98000000,0,0,16300000,0,0,0,0,0,0,0,0,500000,28676024,800000,0,1250000,0,0,350000,0,0,9400000,0,0,0,0,0,0,5500000,0,0,0,69300000,0,0,0,225000,0,0,46800000,0,0,156000000,0,0,0,0,51000000,20000,0,0,30500000,0,43600000,0,51800000,10000000,34100000,4000,1130000,50000,0,0,175000000,0,5000000,5000000,0,0,0,12300000,0,1500000,0,0,27100000,12200000,551000000,0,4900000,4000000,5500000,0,0,8300000,0,0,0,0,0,0,0,14000000,128000000,0,25600000,0,0,0,0,82900000,0,0,0,0,0,0,9650000,3500000,27000000,16000000,14600000,20000000,1000000,15200000,15000000,15500000,35900000,11500000,0,15600000,5000000,0,0,11000000,11200000,21000000,6640000,423000,0,0,0,0,0,0,0,0,0,0,0,0,0,300000,0,0,3200000,0,0,0,0,0,0,0,0,0,0,5000000,0,0,0,0,0,0,25000000,0,0,0,4800000,800000,4050000,0,0,0,7000000,0,0,0,0,4000000,0,0,0,0,0,0,0,18400000,6000000,0,0,0,0,0,0,0,0,4000000,0,0,0,0,30500000,0,137000000,430000,42500000,0,0,0,8400000,0,4700000,7300000,0,0,0,0,44000000,25800000,62800000,7500000,0,0,38400000,0,25500000,0,0,0,0,0,0,0,0,0,0,0,0,3500000,0,0,0,22500000,3000000,0,16400000,0,0,3000000,0,0,0,0,0,0,5920000,0,0,0,0,0,0,116000000,4090000,1000000,20100000,0,350000,10000000,0,30200000,250000,0,0,0,0,0,0,2000000,0,0,0,0,0,0,29000000,0,15200000,0,0,733000,74000000,0,37200000,27800000,0,50000,0,1000000,18600000,0,0,0,8000000,0,20300000,7350000,0,15000000,0,0,0,10200000,0,0,91500000,110000,0,0,0,4000000,0,0,82800000,0,20000000,4000000,0,20300000,0,0,0,0,0,0,108000000,74700000,786000,40000000,0,330877.2,1000000,0,0,0,0,1250000,0,5230000,6500000,0,0,0,8300000,0,10000000,11360117.2,0,0,0,0,0,0,0,6260000,0,450919.35,0,600000,8000000,200000,0,0,0,0,0,0,2620000,100000000,1500000,0,3000000,1102924,0,0,0,0,0,500000,200000,2500000,0,0,0,0,0,0,0,0,40000000,0,0,0,0,0,8000000,1200000,66700000,0,0,0,0,0,0,0,0,0,0,3770000,0,0,100000,0,0,0,0,0,0,0,19500000,0,0,2500000,192000000,60000000,6100000,0,0,0,0,40000,0,0,0,100000000,7480000,0,0,0,0,114000000,0,1000000,0,0,0,0,0,0,0,20000000,0,1930117,1500000,0,0,0,0,0,0,33000000,0,150000,0,0,5539866.3,0,0,0,0,350000,0,0,0,0,0,1000000,0,0,0,500000,0,0,8000000,661754.4,0,51100000,0,0,0,0,0,0,0,1500000,386502.3,0,0,0,0,725000000,0,0,0,183000000,0,0,248000000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,875000,0,100000000,0,0,0,0,0,5955789.6,0,11000000,0,0,0,0,0,22800000,0,0,9600000,0,1600000,0,15000,33000000,11100000,1200000,0,0,1300000,110000,0,0,15300000,11700000,10000000,0,0,2000000,0,0,0,0,0,0,0,600000,1200000,4600000,2000000,0,330877.2,0,28500000,0,0,0,0,0,0,0,0,5200000,0,0,0,0,0,0,0,560000,0,0,0,0,3450000,0,15500000,0,0,0,0,0,500000,0,0,0,0,0,0,0,0,0,0,0,0,33000000,0,0,0,6500000,0,0,0,3000000,0,0,0,0,0,2500000,0,31900000,0,0,1200000,0,0,0,0,0,0,0,0,551462,6000000,0,0,0,0,0,6270000,0,0,0,0,0,0,0,0,0,18500000,61000000,0,0,0,0,4000000,0,25000000,0,0,0,0,0,0,35000,1400000,0,0,0,0,0,5220000,0,0,0,0,0,10000000,1250000,0,0,6950000,0,0,12000000,0,0,0,0,27000000,0,0,0,5340000,0,0,0,0,0,0,0,0,10400000,17300000,0,8750000,0,0,0,60700000,4250000,0,0,0,0,1100000,0,0,76600000,10000000,0,0,2350000,65900000,0,8660000,0,0,10000000,441169.6,0,0,0,0,175000000,0,0,0,0,0,20300000,0,3865023,0,6090000,0,0,2500000,441169.6,0,0,0,0,0,27000000,0,0,0,0,0,5450000,0,32500000,0,0,10000000,0,8100000,0,28500000,0,0,0,0,0,0,0,20000000,0,0,1000000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,7980000,0,0,0,33000000,925000,0,0,0,0,0,0,100000,0,0,34800000,102000000,1600000,1000000,5000000,0,3380000,0,0,200000,0,0,0,0,0,0,0,551462,0,30000,27573100,0,0,21800000,0,0,0,0,0,0,500000,0,1803677.4,0,425000,0,0,0,0,31000000,0,0,0,0,21000000,0,9350000,0,0,34100000,0,0,0,0,6290000,0,0,30000000,493000,0,0,0,0,3000000,0,0,0,0,0,0,0,0,0,0,0,98200000,1000000000,0,44000000,4000000,0,31000000,15600000,0,0,0,5300000,0,0,0,0,0,152000000,16000000,0,0,0,0,0,0,6000000,0,0,0,0,250000,1000000,0,0,22300000,0,0,0,2710000,0,0,14700000,0,0,0,29300000,0,0,0,0,0,0,0,0,250000,7620000,0,0,0,18700000,2900000,0,0,45000000,0,0,49300000,5000000,0,0,350000,0,0,0,5800000,1700000,0,0,500000,0,0,0,0,0,0,5550000,0,0,0,0,43900000,0,13500000,0,0,0,0,0,0,0,0,0,0,1940000,6000000,0,16100000,138000000,61600000,0,11400000,22000000,0,1102924,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,500000,0,0,0,330877.2,0,0,0,19500000,0,0,0,0,0,0,0,1500000,0,0,10700000,0,276000000,6600000,103000000,0,0,53000000,50300000,0,0,0,0,0,0,28500000,0,0,0,12200000,0,0,13400000,127000000,0,0,0,0,0,2700000,38000000,19100000,0,0,0,0,0,0,0,0,0,82000000,0,0,0,0,8200000,0,105000000,0,0,0,1000000,12000000,0,0,0,200000,0,0,0,0,0,0,0,750000,0,0,0,0,0,50000,0,0,0,0,0,0,0,0,0,7250000,0,0,2520000,0,0,0,0,250000,0,0,0,0,0,0,0,2200000,3800000,0,0,300000,275731,0,0,0,19300000,27000000,2500000,0,0,0,0,0,0,0,0,15000000,0,0,0,0,3770000,0,0,0,0,0,0,0,0,0,1213216.4,0,0,0,30000000,150000,0,0,38500000,0,0,6000000,0,0,0,4200000,6000000,0,43400000,0,0,0,0,36300000,0,0,0,37000000,0,12683626,3860234,0,0,200000,0,0,0,0,1700000,18000000,0,7500000,6000000,0,0,0,0,0,0,0,0,0,0,560000,0,0,0,0,0,0,0,369000000,0,3865023,0,0,0,0,0,0,0,0,250000,4900000,70500000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,18000000,0,0,0,14500000,0,0,0,37500000,0,0,0,0,0,42000000,1840000,0,0,5000000,0,0,551462,0,27573.1,55000000,0,0,0,25500000,7000000,80513452,28100000,0,1500000,0,0,0,0,0,37600000,0,0,2000000,0,0,0,0,1650000,2000000,300000,27100000,0,250000,0,0,0,0,0,0,0,1500000,0,0,0,0,0,0,0,26800000,0,0,0,0,0,0,0,0,0,6500000,0,6000000,0,0,0,0,24000000,0,0,0,0,0,0,0,0,1300000,0,0,0,0,0,0,0,0,0,0,1250000,123000000,1680000,6000000,13100000,0,0,19100000,0,9000000,606608.2,0,0,0,0,0,0,0,0,0,0,0,110292.4,4000000,0,0,72900000,2800000,0,0,0,0,0,25036374.8,0,0,0,0,3050000,0,0,5430000,0,0,155000000,53000000,6500000,0,0,0,0,0,4650000,0,0,4700000,90000000,0,12000000,0,0,0,0,0,0,0,0,31000000,0,0,1250000,38900000,0,0,0,0,10000,9400000,0,0,0,0,9900000,55000,0,0,45600000,0,0,7200000,0,0,0,0,0,0,100000,0,0,0,0,0,0,0,1000000,0,350000,0,0,0,0,375000,700000,0,38500000,0,0,5330000,0,0,0,1000000,0,0,0,9400000,0,0,64800000,0,0,0,0,0,0,0,80000,26500000,102000000,0,0,1599239.8,0,0,2500000,0,0,0,0,0,2000000,0,0,0,0,0,0,330877.2,0,0,0,0,0,3010000,0,0,10000000,0,64417.05,15300000,0,0,56500000,0,0,0,0,0,0,0,0,350000,0,0,0,0,11500000,3800000,0,500000,300000,0,0,9100000,600000,0,0,15300000,0,0,0,550000,0,5000000,0,0,68000000,0,0,2300000,0,0,30200000,0,0,0,0,0,0,20900000,15000000,0,0,0,64417.05,0,772046.8,0,0,0,0,0,1000000,0,0,1100000000,0,5000000,0,0,0,0,0,0,0,0,0,29800000,3000000,0,0,0,73000000,0,0,0,0,0,0,10000000,0,0,600000,2600000,1500000,0,0,0,0,0,0,0,0,0,0,2100000,300000,0,0,0,0,0,16000000,0,6000000,0,0,0,0,32500000,0,0,0,0,0,0,0,0,0,0,71200000,0,0,0,0,0,0,0,0,0,70000,0,0,0,0,0,0,0,8660000,0,33700000,245000,51000000,2100000,0,10000000,33900000,0,5140000,0,13500000,1300000,0,800000,1000000,0,10000,0,0,0,0,0,2100000,1102924,0,0,0,0,28100000,0,0,0,0,0,8000000,55500000,0,2900000,0,0,0,0,0,0,3500000,0,0,0,0,0,0,0,0,0,0,0,0,10100000,50700000,0,0,0,755502.94,0,0,0,100000,3800000,550000,1980000,0,25000,0,0,300000,0,0,0,0,0,0,3000000,0,0,0,0,0,0,0,2000000,33200000,0,0,1100000,500000,0,2200000,0,0,0,10000000,0,0,125000,3800000,0,0,0,0,0,0,0,0,0,0,0,0,0,200000,0,0,1300000,0,0,0,0,40000,0,551462,0,0,0,0,0,0,0,0,0,0,0,26200000,0,0,0,0,0,0,0,0,0,0,0,0,0,2205848,0,0,0,0,0,0,0,0,0,0,12700000,0,161000000,0,0,0,0,0,0,0,0,0,0,0,0,16000000,0,0,0,0,28400000,3200000,1000000,31400000,0,0,0,5200000,0,159000000,0,26200000,20000000,0,0,0,0,0,0,0,0,487000000,0,11300000,9000000,6150000,4411696,17536491.6,0,0,0,133987.464,0,0,0,0,0,0,8000000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,6010000,0,0,0,0,40000000,85500000,0,0,35000000,13100000,0,0,0,3000000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,10000000,3308772,2430000,0,0,0,0,0,0,0,0,0,0,20500000,0,0,0,0,3610000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,800000,0,0,0,0,0,38000000,0,0,0,280000,0,15500000,0,7480000,0,0,0,0,500000,0,0,0,0,0,0,0,0,0,0,0,0,120000,25000000,0,0,1710000,22400000,0,1000000,42200000,27500000,0,0,0,0,30000000,0,6000000,44000000,0,0,0,0,0,0,0,0,0,0,0,25000,0,0,0,0,0,5000000,250000,52700000,0,0,0,0,0,0,9900000,25000,80700000,0,17500000,0,0,0,0,0,0,0,0,0,0,0,0,150000,0,0,125000,0,24000000,0,0,0,0,0,0,0,0,36500000,24800000,0,0,0,0,0,0,0,0,50500000,0,8470000,0,0,0,3352888.96,57100000,0,102000000,0,0,0,0,22000000,0,0,0,0,5380000,0,0,0,0,0,0,0,0,0,0,22800000,0,0,1100000,0,0,0,0,0,0,600000,0,1000000,0,0,0,0,0,0,0,0,21500000,0,0,0,0,0,0,0,0,0,0,4730000,0,4700000,0,0,0,0,0,0,0,0,0,0,0,35000000,0,0,1488947.4,0,0,0,2550000,0,0,0,0,2250000,11400000,0,19200000,0,0,0,0,340000,44800000,0,0,0,1440000,4100000,0,0,11000000,0,0,5900000,0,0,0,644170.5,0,0,0,0,0,0,0,0,0,0,0,0,41000000,0,0,33600000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,500000,12000000,0,70000000,0,68000000,34000000,0,5000000,0,0,0,0,0,15000,0,0,5000000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,13300000,0,0,21900000,0,0,0,0,2000000,0,0,0,3000000,0,0,0,0,150000,0,0,0,0,0,0,1800000,0,0,0,0,0,0,0,0,0,15200000,32000000,0,1200000,0,0,11500000,0,0,0,0,41100000,4750000,0,0,0,0,2500000,0,0,15000000,0,0,0,0,0,0,52000000,110292.4,0,10000000,0,0,15000,0,0,0,0,0,0,0,0,21300000,0,0,0,0,0,0,0,0,42500000,8090000,11000000,43000000,15000000,2160000,74000000,13000000,0,0,250000,0,1250000,0,930000,0,0,0,0,0,0,0,0,0,0,120000,20000,0,13300000,329000000,0,0,250000,1000000,0,0,3000000,600000,27800000,0,25000000,0,34080351.6,0,1000000,146000000,1102924,0,0,6250000,0,0,0,515000,370000,0,0,8020000,3270000,1020000,18000,37700000,500000,0,0,0,0,0,0,5390000,0,0,0,9300000,0,0,0,0,0,0,0,0,0,130000,1100000,0,82500000,475000,750000,0,0,0,0,0,33000000,0,0,0,0,330877.2,0,0,0,0,0,0,840000,0,122000000,350000,0,0,0,3100000,0,100000,45000,280000,0,0,0,0,0,1378655,0,0,20500000,0,0,0,0,44900000,1310000,0,800000,0,0,0,0,1000000,15300000,0,250000,3500000,0,0,230000,375000,0,500000,14200000,0,28000000,0,0,1750000,0,24300000,13900000,0,0,0,0,0,150000,0,0,0,34500000,12300000,0,0,225000,0,0,0,0,0,1300000,0,150000,0,0,1000000,0,0,0,0,0,0,0,0,0,0,0,50000,0,0,0,0,0,500000,13300000,12100000,77204.68,0,0,25800000,0,0,0,0,0,0,20000,0,0,0,0,0,0,0,0,0,10600000,0,0,0,0,0,160000,0,100000,0,0,0,0,0,10000000,0,0,23300000,0,0,6000000,4000000,0,26000000,0,0,0,500000,10500000,0,3500000,0,0,0,4200000,0,0,0,0,0,0,0,200000,0,0,0,0,0,1200000,0,0,0,8600000,0,0,0,0,0,0,0,0,16300000,0,0,12000000,0,0,252000000,0,0,0,0,0,0,2300000,0,1000000,0,0,600000,700000,0,0,32000000,83000,0,17000000,0,0,0,0,0,0,0,3840000,0,0,0,0,0,150000,0,0,0,0,0,0,0,1800000,0,0,0,0,0,0,0,0,0,15200000,32000000,0,1200000,0,0,11500000,0,0,0,41100000,0,4750000,0,0,2500000,0,0,0,15000000,0,0,0,0,0,0,0,52000000,110292.4,0,10000000,0,0,15000,0,0,0,0,0,0,0,0,21300000,0,0,0,0,0,0,0,0,42500000,11000000,8090000,43000000,15000000,2160000,74000000,13000000,0,0,250000,930000,0,1250000,0,0,0,0,0,0,0,0,0,0,0,20000,120000,0,13300000,329000000,0,0,0,250000,1000000,0,600000,3000000,27800000,0,0,34080351.6,25000000,1000000,0,146000000,1102924,0,0,6250000,0,0,0,515000,370000,0,0,8020000,3270000,1020000,18000,37700000,500000,0,0,0,0,0,0,5390000,0,0,0,9300000,0,0,0,0,0,0,0,0,0,1100000,82500000,130000,0,750000,475000,0,0,0,0,0,33000000,0,330877.2,0,0,0,0,0,0,0,0,840000,0,0,350000,122000000,0,0,0,0,3100000,100000,45000,280000,0,0,0,0,1378655,0,0,0,20500000,0,0,0,44900000,0,0,1310000,800000,0,0,0,1000000,0,0,15300000,0,3500000,0,230000,375000,250000,0,28000000,500000,14200000,0,0,0,0,1750000,24300000,13900000,0,0,0,0,0,150000,0,0,0,34500000,12300000,0,0,0,225000,0,0,0,0,1300000,0,150000,0,1000000,0,0,0,0,0,0,0,0,0,0,0,0,50000,0,0,0,0,500000,0,13300000,77204.68,12100000,25800000,0,0,0,0,0,0,0,0,20000,0,0,0,0,0,0,0,0,0,10600000,0,0,0,0,0,160000,0,100000,0,0,0,0,0,10000000,0,0,23300000,0,0,6000000,26000000,4000000,0,0,0,0,500000,0,10500000,3500000,0,0,4200000,0,0,0,0,0,0,0,0,0,200000,0,0,0,0,1200000,0,0,0,0,8600000,0,0,0,0,0,0,0,16300000,0,12000000,0,0,0,252000000,0,0,0,2300000,0,0,0,0,1000000,0,600000,0,700000,0,0,32000000,0,83000,0,17000000,0,0,0,0,3840000,0,0,0,0,0,800000,0,1800000,1000000,1500000,0,0,0,0,0,0,4000000,0,46400000,8800000,0,0,45000000,0,1000000,500000,0,2000000,0,0,2000000,0,0,0,9100000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,644170.5,0,0,0,314000,0,10400000,98000,0,0,1700000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1610000,0,0,6000000,0,0,0,0,0,2000000,0,0,0,150000,0,0,0,0,5000000,0,0,0,0,0,0,4500000,0,0,0,0,0,0,0,0,0,0,0,12600000,30500000,0,0,7660000,0,0,0,0,0,0,0,0,0,4180000,0,0,0,0,0,0,0,0,0,0,0,12600000,0,0,17000000,0,0,0,1300000,11700000,16900000,0,0,0,3200000,0,0,0,0,0,0,0,0,0,0,0,0,0,35600000,26400000,0,9000000,0,0,0,75000,0,0,0,700000,22000000,0,30700000,2940000,0,0,0,0,0,0,6400000,0,25500000,33300000,0,0,0,39800000,0,1000000,0,40100000,0,0,0,0,0,0,0,0,0,0,0,0,0,11200000,0,0,0,0,0,0,0,0,0,113000000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,915000,0,3700000,28000000,69900000,15000000,0,0,0,125000,0,0,3650000,0,0,0,0,0,0,0,92000000,0,0,0,0,0,3750000,17000000,0,0,11500000,0,0,0,15000000,551462,0,0,20000,0,0,0,0,1300000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3180000,48000000,950000,200000,0,0,0,0,0,11600000,0,0,0,0,1930117,49900000,0,2920000,0,0,0,0,0,0,0,211000000,0,0,0,0,0,0,0,0,10600000,0,50000000,0,0,0,0,0,0,0,0,0,0,3900000,0,2190000,246000000,0,0,0,0,0,0,0,0,0,0,0,0,0,5000000,7520000,0,0,0,24000000,0,0,0,0,0,0,0,0,0,141000000,0,60000,0,1400000,35000000,4000000,59500000,0,3500000,23492281.2,0,107000000,0,0,0,0,0,1000000,0,13800000,789000000,0,0,25000,937485.4,0,0,0,1500000,0,0,0,0,14300000,0,0,5000000,0,1300000,0,0,0,0,0,0,0,3100000,0,16000000,0,0,0,0,0,0,0,0,0,1600000,0,0,350000,0,0,2757310,0,0,0,0,0,26200000,0,0,0,0,0,0,10100000,1200000,0,46500000,0,0,0,0,0,0,0,0,0,0,0,0,0,13000000,0,0,0,0,0,0,0,0,1764678.4,1000000,0,0,25000,0,0,0,0,118000000,0,0,0,0,0,200000,0,500000,0,0,0,0,3000000,0,300000,0,0,5900000,0,3639649.2,0,0,0,0,0,0,4000000,0,5300000,0,550000,0,0,0,0,0,0,10200000,0,2500000,0,0,61100000,24000000,0,0,0,0,551462,0,0,2000000,0,0,0,0,12900000,940000,0,0,0,0,0,0,0,0,0,0,1654386,0,0,0,0,7000000,0,0,0,0,0,1300000,0,0,0,0,0,0,0,0,0,0,51800000,63600000,0,0,0,0,0,45000000,7550000,0,0,0,505000,37800000,0,0,750000,0,0,0,0,0,0,649622.236,0,0,300000,0,0,0,350000,18300000,0,0,0,0,0,0,0,0,1147040.96,0,0,0,0,0,0,22500000,0,5700000000,0,0,0,0,0,0,0,0,0,0,0,24500000,0,0,2000000,1700000,3000000,39800000,50000,0,0,0,0,0,0,0,0,0,0,0,0,0,17000000,0,0,0,0,0,650000,18500000,0,0,0,0,0,0,0,0,1000000,0,0,0,0,0,0,2750000,0,0,0,0,0,0,0,0,30300000,0,0,5380000,0,0,0,0,0,0,2300000,0,0,0,0,1000000,0,0,0,0,0,0,14000000,0,100000,0,49600000,0,18400000,0,100000,0,0,0,27500000,0,0,22500000,0,0,0,43500000,0,0,3900000,0,0,0,0,0,86700000,0,0,0,0,0,350000,30000000,68400000,67900000,41500000,190000,41300000,0,0,0,0,0,0,0,0,0,2000000,0,14500000,7500000,600000,55146.2,0,30200000,0,0,0,34700000,0,0,0,0,0,0,0,0,2500000,0,0,15000,0,0,0,0,0,0,0,0,0,539000,64417.05,500000,1830000,0,0,104000000,0,0,0,0,0,0,0,10000,0,0,0,0,0,0,0,0,0,0,0,25000,65000,0,0,26900000,0,0,0,0,510000,0,0,0,0,0,0,0,0,4000000,0,0,0,30500000,12000000,0,165438.6,0,0,0,0,0,0,24300000,0,0,0,0,0,0,0,0,0,0,0,110000,0,0,0,250000,0,20000000,705871.36,0,0,0,0,0,150000,0,0,0,0,0,0,0,5400000,0,1310000,0,0,441169.6,0,50000000,0,0,0,0,0,0,1000000,0,0,0,0,4000000,0,0,0,0,0,500000,50000,0,0,0,0,0,0,0,0,12700000,0,0,0,0,0,0,0,0,0,600000,0,0,0,0,0,0,0,0,750000,650000,750000,0,0,0,0,0,0,0,0,0,0,0,0,0,655000,2370000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,110000000,0,0,0,0,0,0,0,0,0,125000,0,0,350000,0,0,0,0,10300000,0,0,0,0,0,30400000,0,28300000,0,0,0,963000,0,2500000,0,0,0,0,0,0,100000,0,0,0,0,0,29100000,0,40300000,0,110292.4,0,0,0,0,0,0,0,0,0,20700000,128834.1,10000000,0,0,0,2700000,0,0,0,0,0,69500000,0,0,0,0,200000,0,345000,0,0,0,0,10000000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,42500000,0,0,0,0,0,0,0,0,0,0,800000,0,0,0,23000000,0,0,0,0,0,0,0,0,0,250000,0,15000000,31000000,0,0,0,1690000,0,0,0,0,13600000,0,0,0,0,0,0,0,5000000,0,0,18700000,49000000,51000000,0,0,0,0,0,0,0,0,0,6000000,0,250000,0,0,0,5540000,0,0,0,0,0,0,0,0,0,0,0,0,0,24200000,0,0,0,0,22000000,7900000,0,9400000,0,22100000,2000000,5000000,2700000,0,0,0,0,0,0,0,0,0,0,0,4200000,0,0,0,0,500000,60000000,1100000,0,0,0,0,0,0,0,0,0,15000,0,0,2172051,1770000,0,250000,0,12700000,44800000,0,0,0,26800000,0,0,0,0,38300000,0,0,0,3800000,0,0,0,0,0,0,0,0,2000000,0,0,0,0,0,0,0,0,0,0,0,10000,0,0,0,0,0,0,0,0,0,0,218000000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,14500000,0,0,0,0,0,0,0,0,0,250000,0,600000,0,0,30500000,0,0,1400000,43500000,0,8600000,250000,0,34900000,212000000,8600000,0,0,150000,0,0,4700000,0,0,0,0,0,2000000,0,0,132000000,1500000,0,0,0,0,5900000,0,9160000,54000000,59000000,0,0,0,0,26500000,17600000,0,49900000,18000000,8000000,4410000,750000,11800000,0,0,0,0,0,0,0,0,0,0,0,0,0,1500000,1170000,16000000,56700000,0,16104262.5,0,0,1000000,250000,52000000,0,0,0,0,0,0,0,10700000,27600000,0,59000000,0,0,0,0,0,0,0,59000000,1102924,7720000,0,0,21100000,6210000,2620000,0,0,15200000,0,5000000,1400000,12000000,0,0,0,0,0,0,0,0,0,0,0,0,250000,0,0,0,40000000,96400000,312000000,41500000,0,22000000,3480000,0,0,0,0,5000000,8000000,13000000,0,0,21500000,130000000,65000000,0,0,284000000,5500000,17000000,42600000,155000000,11400000,24100000,20000000,0,27300000,1950000,0,48000000,3170000,0,2800000,0,0,0,2000000,0,0,0,0,0,0,0,0,836000000,0,17000000,0,0,0,0,0,5290000,50000000,0,0,0,0,0,0,0,104000000,2000000,0,1080000,0,0,0,3000000,12000000,1500000,24500000,0,0,0,8330000,84300000,3000000,0,0,15000000,20000000,60000000,17500000,250000,0,101000000,0,0,0,12400000,15800,0,0,0,1400000,65800000,26000000,7450000,164000000,0,0,11000000,0,1700000,0,13900000,20700000,0,12000000,24000000,20000000,0,0,24000000,37900000,17400000,13200000,10900000,10000000,173000000,35200000,40100000,158000000,136000000,57600000,18000000,0,74300000,59000000,0,0,0,0,7000000,50600000,0,0,0,7000000,0,12500000,0,0,0,0,8860000,0,38000000,0,410000000,0,0,5000000,35400000,31000000,25000000,6080000,0,0,0,0,0,44300000,2000000,0,0,111000000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4480000,0,0,0,0,0,0,0,0,23000000,0,84500000,0,0,0,12800000,0,0,10000000,28400000,15000000,8000000,35000000,500000,26100000,29500000,0,0,72100000,2000000,1500000,0,0,86700000,25000000,0,58900000,84500000,0,0,25500000,12500000,10000000,29000000,50400000,3500000,53800000,0,43000000,0,0,0,17100000,0,0,0,0,0,0,0,0,0,150000,0,0,0,3723305.49,58000000,0,0,0,0,4940000,0,0,0,0,0,0,0,0,0,0,12200000,6500000,11600000,31000000,0,0,0,0,31500000,0,0,0,64900000,0,0,0,325000,1460000,20500000,0,0,0,3100000,49000000,0,0,28300000,0,0,7000000,0,0,0,0,7970000,22000000,0,0,0,0,0,0,0,0,0,0,0,0,0,10600000,27000000,0,0,0,0,0,614538.657,45000000,0,90000,0,0,0,0,0,0,10200000,7000000,0,0,10500000,30000000,0,0,22000000,0,0,0,0,0,0,0,0,0,6700000,1200000,0,0,0,5700000,0,5000000,4700000,0,0,15600000,0,25000000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,27400000,0,0,0,10800000,10600000,0,21700000,0,23000000,16200000,6000000,0,0,26200000,0,0,40000000,10000000,0,0,7100000,16300000,0,0,0,0,50000,4000000,2000000,13200000,4580000,0,0,33200000,0,58200000,2600000,0,13600000,0,0,0,0,2205848,0,0,0,0,0,0,0,7700000,0,17300000,2000000,42000000,99400000,2600000,0,0,0,392000,90100000,12000000,0,5000000,14000000,10300000,0,0,44400000,7000000,0,0,3000000,18200000,77000000,0,0,0,0,0,13000000,12000000,13000000,0,1400000,18000000,0,0,55900000,0,0,800000,20200000,7000000,0,0,26800000,7100000,0,0,20000000,0,30000000,0,20000000,3000000,0,12000000,80000000,0,0,0,0,0,0,0,5000000,0,0,0,0,0,80000000,0,28000000,0,0,0,0,0,0,0,0,0,18700000,0,3470000,0,23600000,0,0,0,0,0,30000000,27500000,0,5890000,0,0,0,0,68400000,0,0,0,70600000,550000,0,30800000,95000000,0,0,0,32300000,2300000,36000000,32700000,0,0,600000,14800000,0,0,0,0,0,1500000,0,0,0,0,0,0,641000,0,3540000,9000000,0,0,70300000,10100000,0,0,0,0,0,0,0,18000000,0,13800000,49000000,43000000,30800000,7630000,0,0,10000000,8500000,0,375000,7000000,15000000,0,0,55500000,2150000,44000000,0,0,45100000,0,32000000,30600000,0,42000000,23700000,600000,5800000,0,0,0,500000,0,15000000,0,0,0,0,4500000,10000000,8500000,8800000,0,0,53000000,30800000,4500000,40700000,14000000,11700000,1100000,7760000,0,0,0,0,0,0,0,200000,27000000,0,0,0,0,0,0,16300000,225000,0,0,3000000,0,0,0,0,0,1500000,43400000,0,72000000,29300000,38800000,0,3100000,0,0,0,0,0,9868692.06,0,0,0,0,142000000,30000000,0,0,0,0,113000000,1820000,0,0,0,0,30000000,4000000,0,48100000,3000000,0,66700000,9000000,0,0,2500000,0,0,0,0,0,9000000,0,0,0,0,0,0,0,0,8800000,12800000,0,30000000,100000,1200000,0,0,0,42000000,0,0,0,0,7500000,0,0,0,0,0,7000000,4300000,0,1000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,24500000,45000000,0,5000000,0,1200000,6250000,44500000,0,11300000,0,10000000,0,0,1450000,0,12500000,638000,0,4411696,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,23400000,6530000,0,0,155000,0,0,0,0,0,7500000,0,2300000,0,7000000,0,0,0,0,0,0,0,0,0,0,19400000,0,0,50000,0,0,0,0,2000000,27000000,4000000,600000,0,0,0,0,0,0,0,27000,0,0,0,0,0,1580000,0,0,0,0,4000000,4200000,2000000,0,0,0,0,0,0,100000,0,0,0,0,0,0,0,0,0,0,198526.32,0,0,0,0,0,0,250000,0,1500000,0,0,0,8000000,0,0,3860234,2290000,0,0,0,0,0,0,0,0,0,0,0,331000000,0,1200000,0,0,0,0,8010000,0,0,35300000,1300000,0,0,0,0,467639.776,23000000,28500000,0,0,0,0,1000000,0,0,0,0,0,0,0,0,69000000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,125000000,0,5000000,0,0,0,0,143380.12,700000,0,220584.8,1764678.4,827193,0,0,0,0,0,17500000,0,4800000,0,0,50300000,0,11500000,0,0,5500000,0,0,0,0,0,0,55800000,0,0,1400000,0,0,223893.572,0,0,0,0,0,15000000,0,0,0,0,30000,0,0,15100000,0,0,2000000,300000,0,100000,0,0,0,0,0,90000,0,2380000,0,10500000,0,0,0,0,18800000,760000,0,0,0,0,0,0,8250000,9000000,0,0,0,0,0,37000000,2000000,0,0,3000000,0,0,0,0,0,0,0,0,0,0,0,0,7950000,0,0,0,0,5790000,0,0,0,0,6230000,14500000,0,0,15000000,0,0,10800000,0,0,0,0,0,0,0,270000,0,0,0,0,0,0,10200000,0,57300000,0,0,0,0,911000,0,0,0,926000,0,0,0,150000,0,352000,0,9000000,0,0,200000,8000000,100000,0,0,52100000,0,13000000,2000000,0,0,27573100,0,5180000,95000,0,0,0,0,0,0,0,0,0,0,0,0,62500000,0,6700000,322000000,250000,0,4340000,0,0,24100000,0,0,0,0,0,0,0,21386460.6,0,0,122000000,0,0,10938015.09,301000000,0,100000,0,937485.4,150000,0,0,0,24100000,100000,0,0,44700000,3400000,11900000,0,50000,0,0,42257584.8,0,2898767.25,0,4818395.34,0,72800000,0,8300000,0,0,0,0,0,0,150000,0,0,0,0,0,0,0,67400000,0,0,0,0,10000000,0,0,0,0,0,0,0,100000,0,0,0,0,0,0,3200000,0,16300000,14700000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,180000,0,0,9950000,1546009.2,0,0,0,0,0,0,0,0,0,10000000,10000,0,0,1130000,0,0,0,0,500000,0,21000000,40800000,51400000,0,3000000,0,3308772,600000,0,0,0,64200000,0,0,0,0,0,0,7300000,0,1000000,0,0,400000,0,0,0,0,0,1000000,4030000,3640000,0,250000,0,0,0,0,0,0,125000,0,0,0,0,30000,0,18100000,6500000,300000,0,155512.284,0,0,40000000,0,0,0,0,0,0,0,0,500000,0,5100000,16000000,0,0,0,0,0,0,0,0,4380359.4,100000,0,0,2350000,0,0,0,0,110292.4,0,0,10400000,0,7730046,0,0,500000,0,0,0,0,0,0,0,0,0,0,1130000,0,0,0,7900000,0,0,41800000,0,39500000,3140000,65000000,0,0,14500000,0,0,0,0,80100000,0,0,0,0,0,0,0,0,2500000,0,0,6000000,34500000,0,0,0,0,0,0,0,224000000,0,0,190000000,30000000,0,0,0,0,0,0,0,0,0,0,0,0,19100000,17000000,0,55146.2,0,0,0,3000000,0,0,0,0,0,0,0,310000,80000,0,0,5000000,0,25000000,119000000,0,0,1100000,0,100000,0,0,30000,0,112000000,0,23900000,7100000,10000000,0,0,47800000,0,110000000,0,11000000,100000,7000000,57600000,42300000,0,10000000,0,0,0,0,0,0,1155641.877,12400000,1213216.4,20000000,0,0,0,0,0,0,0,0,772046.8,103067.28,20000000,0,9926316,0,0,0,44500000,5850000,0,0,0,0,0,0,0,0,28800000,0,0,1210000,0,0,0,3000000,0,0,100000,0,0,0,32000000,368376616,44000000,0,0,3440000,300000,0,3200000,0,0,0,0,0,0,400000,0,0,0,0,44500000,0,0,0,94000000,58000000,8800000,8000000,250000,0,0,13000000,0,0,0,0,0,0,0,0,1350000,0,0,0,0,0,0,0,0,0,73000000,0,0,0,0,0,0,0,0,795000,0,18700000,1288341,0,14100000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,400000,28896608.8,0,0,0,1200000,0,0,0,42000000,0,0,0,0,0,40000,0,0,7000000,0,4950000,0,5000000,0,0,0,0,4700000,0,0,0,0,0,0,0,0,0,0,0,4360000,0,0,22500000,64000000,0,0,0,0,827193,0,385000,0,0,0,0,0,0,0,0,0,0,220584.8,0,0,0,0,0,0,0,3230000,890000,0,0,0,0,0,0,0,0,0,0,0,19500000,0,16000000,0,0,0,10800000,0,0,0,0,0,0,0,0,0,459000000,1770000000,91500000,294000000,85100000,57000000,0,0,39300000,0,0,800000,0,0,46300000,500000,0,0,100000000,0,222000000,0,4130000,0,0,0,0,0,151000000,53400000,0,0,30000000,81000000,156000000,27500000,89500000,65200000,11500000,3010000,800000,0,0,0,0,0,0,0,0,0,0,0,11000000,0,1020000,41800000,0,0,1820000,0,0,17000000,0,0,0,0,25000000,0,0,0,0,17400000,0,310000,0,0,0,39200000,0,0,13400000,45400000,68800000,27000000,0,0,0,7000000,0,0,5460000,17000000,42800000,9350000,9300000,16100000,0,0,0,0,0,0,0,0,7500000,0,0,500000,8260000,13000000,0,0,70200000,20600000,4130000,78000000,0,34000000,19000000,0,100000,0,0,14000000,55400000,19200000,0,0,0,2200000,0,0,0,0,0,0,7720000,89500000,0,0,3400000,0,6000000,0,0,0,42200000,79800000,5600000,5500000,0,0,7000000,26000000,6000000,0,0,0,0,0,25900000,49500000,11000000,0,0,54400000,240000000,0,0,0,7580000,0,4000000,5720000,5000000,0,8000000,4000000,3860000,9050000,0,0,7710000,0,20000,0,0,0,275000,24700000,9470000,0,3500000,0,6000000,0,600000,0,0,0,0,0,3020000,0,3000000,6660000,6500000,11800000,1800000,0,32208.525,2000000,6900000,9100000,0,0,0,0,0,0,8660000,3200000,0,23300000,0,61000000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1750000,21700000,1680000,2029380.16,1600000,0,1560000,0,0,0,0,726000,99500000,0,0,485000,1102924,35200000,692000,0,1550000,0,651000,1170000,2400000,117000000,0,12100000,0,8030000,1300000,1190000,1080000,0,0,11000000,13800000,1040000,1020000,1000000,750000,0,0,496000,2980000,400000,5000,3370000,39900000,0,909000,643004.692,0,237128.66,1630000,1660000,17500000,0,794000,0,60700000,0,10000,0,12300000,1200000,21000000,0,3800000,0,2500000,0,120000,0,457000,0,0,0,9910000,10000000,0,0,0,0,0,0,50000,0,100000,0,0,650000,0,0,3420000,6790000,0,45000000,200000000,0,0,0,19500000,0,0,0,18000000,641000000,0,89000000,11500000,0,0,38602.34,110292.4,0,0,0,0,0,0,0,0,0,0,10000000,337000000,8000000,7920000,0,0,9500000,98400000,0,11400000,0,2371286.6,9000000,0,0,37000000,0,0,6500000,0,6000000,6800000,57700000,5000000,28000000,55800000,26500000,29000000,0,0,0,0,0,0,0,0,0,0,4030000,13700000,20600000,7100000,0,29000000,0,29100000,22200000,14000000,800000,31000000,15100000,0,6250000,0,0,0,29000000,186000000,128834100,3750000,3160000,0,53000000,0,27000000,15600000,43500000,18400000,0,5000000,0,0,0,41500000,0,0,0,83700000,0,3000000,600000,2300000,13400000,15700000,15100000,38000000,0,3000000,24300000,86200000,1500000,8350000,0,0,0,0,0,0,0,500000,0,0,0,0,7450000,1250000,253000000,12900000,2500000,611000,387000,0,0,58600000,0,0,51800000,0,0,0,0,0,0,0,0,0,0,0,0,0,600000,51000000,0,4710000,500000,3890000,1990000,8000000,0,60500000,0,0,0,0,0,3900000,0,0,0,1710000,2750000,11029240,4500000,0,0,85000000,35000000,16000000,0,0,0,16800000,0,0,0,0,0,7300000,0,1500000,33800000,0,15500000,0,0,50000,0,0,0,0,0,0,0,8205754.56,18000000,28100000,11100000,0,0,127000000,50000000,10000000,9000000,0,0,49000000,0,0,0,1200000000,11600000,0,0,0,0,41300000,29000000,0,0,0,15900000,0,45219884,0,0,70000000,0,0,0,0,0,39100000,7600000,0,0,0,0,0,0,0,4200000,0,0,0,57000000,10900000,0,0,0,0,13000000,0,0,5500000,2600000,9700000,99500000,0,0,1600000,0,0,43700000,40600000,0,0,94800000,0,106000000,146000000,7500000,0,0,0,0,0,7210000,175000,16000000,0,57900000,17500000,84900000,0,0,0,0,0,40800000,189000000,0,0,78100000,0,18000000,19000000,50600000,350000000,13600000,68800000,0,330877.2,1654386,8000000,1000000,0,0,42000000,0,0,13300000,169000000,100000,7000000,0,0,89500000,52300000,801000,108000000,23000000,357000000,0,0,21200000,55000000,77200000,0,111000000,0,100000000,0,155000000,0,3000000,1000000,58000000,1200000,62000000,17000000,7750000,29300000,0,129000000,20000000,0,56000000,87000000,0,12000000,61900000,13700000,0,16500000,48200000,0,0,0,0,100366.084,77400000,0,0,0,2576.682,28800000,42500000,0,0,275731,0,5970000,16900000,145000000,0,42100000,45700000,0,0,0,42500000,192000000,0,1288341,0,0,0,7500000,0,0,30500000,0,107000000,39500000,0,116000000,0,11800000,0,52800000,17000000,1570000,182000000,8970000,10400000,2000000,0,53700000,12000000,14300000,31700000,3790000,0,20700000,0,16000,6000000,2500000,2000000,2320000,2410000,0,13400000,0,0,0,4500000,0,24000000,40000000,70600000,30700000,5000000,78800000,28676024,3419064.4,68100000,0,53000000,54300000,0,5000000,10600000,2640000,0,1102924,3200000,103000000,98900000,275731,0,0,172000000,0,33500000,81300000,115000000,110000000,600000,500000,0,19400000,10100000,51400000,5000000,0,0,8000000,87000000,0,0,95900000,31800000,80500000,0,17900000,5730000,0,800000,0,11300000,16000000,220584.8,0,63000000,8270000,0,62600000,0,5500000,0,34700000,0,0,0,15000000,40000000,35200000,0,23000000,0,41000000,0,52400000,0,0,29400000,0,29800000,0,0,155000000,0,0,37400000,18000000,0,77500000,4000000,55000000,0,0,0,0,10000000,0,0,0,0,103000000,0,0,0,0,0,0,0,0,0,0,316000000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,80000000,0,0,0,0,0,0,0,0,0,0,0,7500000,50000000,23900000,3500000,43500000,21500000,12000000,33100000,15000000,0,11000000,232000000,0,28000000,25000000,0,0,0,0,9000000,17000000,0,0,551462,0,39000000,0,0,0,0,0,0,0,6800000,0,0,0,10800000,17000000,0,0,0,0,25000,0,0,0,0,0,3300000,23000000,0,46800000,34000000,136000000,11800000,126000000,7670000,8000000,26000000,7450000,9130000,9000000,19000000,32000000,0,0,18100000,9910000,8760000,0,0,2000000,2300000,0,15500000,0,12700000,40000000,57000000,1250000,3639649.2,48100000,0,38000000,26500000,0,0,17000000,0,55300000,74600000,3000000,2780000,7300000,18036774,0,2060000,9000000,0,1654386,34800000,0,12000000,0,8500000,32500000,26600000,86200000,47000000,458000000,5100000,13300000,0,4600000,0,33200000,0,0,0,0,1700000,0,0,0,0,0,0,0,0,0,0,1500000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2200000,0,0,0,0,0,0,0,0,0,0,0,24200000,0,0,0,26800000,0,0,12000000,1000000,0,0,0,0,12000000,0,0,0,75000,0,116000000,0,0,0,0,0,0,0,0,0,0,0,16000000,22500000,2710000,1288341,28100000,5000000,1102924,1654386,0,1500000,0,1840000,0,0,0,0,0,37000000,0,96800000,26300000,12700000,30000000,0,0,0,0,57700000,0,0,0,0,0,0,0,0,69000000,20000000,0,86200000,0,3500000,2000000,0,1000000,0,0,0,0,0,941000,0,0,28000000,0,7380000,0,20000,0,2000000,2930000,0,0,2560000,0,0,0,0,0,0,0,12500000,0,551462,0,1110000,1570000,0,0,0,0,14500000,0,0,106000000,0,0,0,0,0,0,0,0,47500000,0,0,0,0,5000000,3000000,0,320000,5500000,0,20000000,22000000,27700000,35000000,1700000,0,0,0,100000,0,0,54000000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,100000,10000000,0,630000,0,0,42400000,25200000,46000000,10000000,28000000,0,4800000,0,0,0,0,0,0,0,0,7000000,9500000,1960000,1670000,0,1720000,0,0,7840000,0,0,0,0,7300000,0,325000,0,900000,0,29000000,0,0,0,0,6760000,49500000,0,0,0,0,1580000,0,0,2810000,0,0,0,0,0,39000000,0,0,3000000,7500000,0,0,0,0,0,0,0,0,0,0,0,7000000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4080000,0,0,0,0,0,552000,250000,0,0,0,0,0,0,0,0,16600000,0,7400000,1102924,0,0,28300000,0,0,0,0,100000000,51000000,17200000,0,0,0,0,0,0,13000000,0,7800000,0,136000000,6500000,0,0,0,0,0,0,0,0,0,121000000,2000000,2000000,0,62700000,11000000,1246304.12,0,0,0,0,0,0,0,0,0,0]
# companies_df_final['total_money_raised_USD'] = [float(e) for e in money_to_USB]

In [84]:
print(min([e for e in companies_df_final.total_money_raised_USD if e!=0]))
print(max([e for e in companies_df_final.total_money_raised_USD]))

1000.0
5700000000.0


In [85]:
companies_df_final.to_csv('./input/companies_df_final.csv', index=False)

In [86]:
coll2 = db['companies_cleaned']

In [87]:
companies_records = companies_df_final.to_dict('records')
# db.insert_many(df.to_dict('records'))
# https://stackoverflow.com/questions/20167194/insert-a-pandas-dataframe-into-mongodb-using-pymongo

In [89]:
coll2.insert_many(companies_df_final.to_dict('records'))

In [88]:
companies_records

[{'_id': ObjectId('52cdef7c4bab8bd675297d8b'),
  'name': 'AdventNet',
  'founded_year': 1996.0,
  'category_code': 'enterprise',
  'total_money_raised': 0.0,
  'deadpooled_year': 1998.0,
  'num_offices': 1,
  'office_1_longitude': -121.904945,
  'office_1_latitude': 37.692934,
  'office_1_location': {'type': 'Point',
   'coordinates': [-121.904945, 37.692934]},
  'office_1_city': 'Pleasanton',
  'office_1_state_code': 'CA',
  'office_1_country_code': 'USA',
  'office_2_longitude': nan,
  'office_2_latitude': nan,
  'office_2_location': {},
  'office_2_city': None,
  'office_2_state_code': None,
  'office_2_country_code': None,
  'office_3_longitude': nan,
  'office_3_latitude': nan,
  'office_3_location': {},
  'office_3_city': None,
  'office_3_state_code': None,
  'office_3_country_code': None,
  'office_4_longitude': nan,
  'office_4_latitude': nan,
  'office_4_location': {},
  'office_4_city': None,
  'office_4_state_code': None,
  'office_4_country_code': None,
  'office_5_longitu